In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import KY_utils.KY_baseline_functions as base

# restore saved variables
%store -r summary_M2_ky_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_M_two_year'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_M_two_year'].values

### Model

In [3]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Random Forest
n_estimators =  [40,50]
depth = [8,9]
rf_summary = base.RF(KY_X, KY_Y, FL_X, FL_Y,
                     depth=depth, 
                     estimators=n_estimators, 
                     seed=816)

#### XGBoost
learning_rate = [0.01]
depth = [7]
n_estimators = [200]
xgb_summary = base.XGB(KY_X, KY_Y, FL_X, FL_Y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       seed=816)

#### save results
summary_M2_ky_model = {"Logistic": logistic_summary,
                                    "Lasso": lasso_summary,
                                    "LinearSVM": svm_summary,
                                    "RF": rf_summary,
                                    "XGBoost": xgb_summary}
%store summary_M2_ky_model

Stored 'summary_M2_ky_model' (dict)


In [4]:
results = []

for model_name, model_summary in summary_M2_ky_model.items():
    results.append([model_name, 
                    model_summary['best_auc'], 
                    model_summary['auc_diff'], 
                    model_summary['best_param'], 
                    model_summary['FL_score']])
results

[['Logistic',
  0.6720695697967902,
  0.001060528874191169,
  {'C': 0.1},
  0.5394036250964965],
 ['Lasso',
  0.6721094450041449,
  0.0009490095656821129,
  {'C': 0.1},
  0.5390863762782484],
 ['LinearSVM',
  0.6718673264949555,
  0.000869292704615332,
  {'C': 1},
  0.5393177035415543],
 ['RF',
  0.6807706505082579,
  0.0203593248187941,
  {'max_depth': 9, 'n_estimators': 50},
  0.566778232501084],
 ['XGBoost',
  0.6859445504564078,
  0.020054872146181624,
  {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 200},
  0.549892664149826]]

In [5]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\baselines\\Two Year\\"
results = [["Misdemeanor",             
            round(logistic_summary['FL_score'],3), 
            round(lasso_summary['FL_score'], 3), 
            round(svm_summary['FL_score'],3), 
            round(rf_summary['FL_score'],3), 
            round(xgb_summary['FL_score'],3)]]
with open(path + 'Two Year KY_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)